In [3]:
import os
import asyncio
import nest_asyncio
import BinanceBackTest
import numpy as np
import utils

nest_asyncio.apply()

symbols = ['BTCUSDT', 'ADAUSDT', 'XRPUSDT']
intervals = ['1m', '5m', '1h']
start_date = '2024-01-01'
end_date = '2024-08-28'
obj = BinanceBackTest.DataManager(symbols=symbols,
                                  intervals=intervals,
                                  start_date=start_date,
                                  end_date=end_date)
kline_data = asyncio.run(obj.generate_kline_interval_data(save=True))
kline_data = utils._convert_to_array(kline_data=kline_data)
closing_data = obj.generate_kline_closing_sync(kline_data=kline_data, save=True)

In [ ]:
kline_data = asyncio.run(obj.generate_kline_interval_data())
kline_data = utils._convert_to_array(kline_data=kline_data)

In [ ]:
objkline_data

In [ ]:
np.sum(output_data[symbol]['1m'][:4], axis=0) -output_data[symbol]['5m'][3]

In [ ]:
np.set_printoptions(suppress=True, precision=20)
n = 0
# kline_data.round(decimals=3)
for i in range(4, 10000, 5):
    data = output_data[symbol]['5m'][i]-kline_data[symbol]['5m'][n]
    n +=1
    print(data[8])
    print(f'n - {n}')

In [ ]:
data[8]